In [1]:
pip install torchsummary


Note: you may need to restart the kernel to use updated packages.


In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from tqdm import tqdm
from torchsummary import summary

In [3]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        #Fresh start
        self.conv1  = nn.Conv2d(1, 8, 3, padding=1)         #inpu:1x28x28,  output:8x28x28      RF:3x3
        self.bn1    = nn.BatchNorm2d(8)
        self.conv2  = nn.Conv2d(8, 12, 3, padding=1)        #input:8x28x28  output:12x28x28     RF:5x5
        self.bn2    = nn.BatchNorm2d(12)
        self.pool1  = nn.MaxPool2d(2,2)                     #input:12x28x28 output:12x14x14     RF:10x10

        self.conv3  = nn.Conv2d(12, 16, 3, padding=1)       #input:12x14x14 output:16x14x14     RF:12x12
        self.bn3    = nn.BatchNorm2d(16)
        self.conv4  = nn.Conv2d(16, 20, 3, padding=1)       #input:16x14x14 output:20x14x14     RF:14x14
        self.bn4    = nn.BatchNorm2d(20)
        self.pool2  = nn.MaxPool2d(2, 2)                    #input:20x14x14 output:20x7x7       RF:28x28
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.fc1 = nn.Linear(20*7*7, 10)
        #self.fc2 = nn.Linear(20,10)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)

        x = self.relu(self.bn3(self.conv3(x)))
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)

        x = x.view(-1, 20*7*7)
        x = self.fc1(x)
        
        return F.log_softmax(x)

use_cuda = torch.cuda.is_available()
#device = torch.device("cuda" if use_cuda else "cpu")
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = Network().to(device)
#summary(model, input_size=(1, 28,28))

network = Network()
total_parameters = sum(p.numel() for p in network.parameters())
print(f"Total Parameters : {total_parameters}")
print(network)

mps
Total Parameters : 15522
Network(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(8, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(12, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(16, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (fc1): Linear(in_features=980, out_features=10, bias=True)
)


In [4]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        #  transforms.RandomApply([transforms.CenterCrop(22)], p=0.1),
                        #  transforms.Resize((28, 28)),
                         #transforms.RandomRotation((-15., 15.)),
                         transforms.RandomRotation(15),
                        # transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        #loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

model = Network().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(0, 19):
    print(epoch+1)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

1


  0%|          | 0/469 [00:00<?, ?it/s]/var/folders/w9/1gz_7_dn4zs29tysldrxvfnh0000gn/T/ipykernel_58751/3977870918.py:34: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.039021268486976624 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 29.64it/s]



Test set: Average loss: 0.0597, Accuracy: 9807/10000 (98.07%)

2


loss=0.06646855920553207 batch_id=468: 100%|██████████| 469/469 [00:09<00:00, 47.71it/s] 



Test set: Average loss: 0.0319, Accuracy: 9907/10000 (99.07%)

3


...


Test set: Average loss: 0.0233, Accuracy: 9929/10000 (99.29%)

18


loss=0.01126453559845686 batch_id=468: 100%|██████████| 469/469 [00:09<00:00, 48.00it/s]  



Test set: Average loss: 0.0224, Accuracy: 9935/10000 (99.35%)

19


loss=0.01486246194690466 batch_id=468: 100%|██████████| 469/469 [00:09<00:00, 47.68it/s]  



Test set: Average loss: 0.0242, Accuracy: 9925/10000 (99.25%)

